# "SRAG Data"
> "Data from opendatasus"

- toc: true
- branch: master
- badges: true
- comments: false
- categories: [srag, covid, opendata, sus]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

Dicionário de dados:

- [2020](https://opendatasus.saude.gov.br/dataset/ae90fa8f-3e94-467e-a33f-94adbb66edf8/resource/8f571374-c555-4ec0-8e44-00b1e8b11c25/download/dicionario-de-dados-srag-hospitalizado-27.07.2020-final.pdf)

- [2021](https://opendatasus.saude.gov.br/dataset/9f76e80f-a2f1-4662-9e37-71084eae23e3/resource/b3321e55-24e9-49ab-8651-29cf5c8f3179/download/dicionario-de-dados-srag-hospitalizado-27.07.2020-final.pdf)

In [1]:
#hide

import os.path
import pandas as pd
import ssl
from urllib.request import urlopen
from bs4 import BeautifulSoup

# import matplotlib.pyplot as plt
import altair as alt

In [2]:
#hide

def get_last_bd_srag_csv_url(year=2021):
    
    available_years = (2020,2021)
    if year not in available_years:
        print('year not valid. Available years:',available_years)
        return
    
    # Se nao achar, retorna última url encontrada
    srag_url = f'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/{year}/INFLUD-29-03-2021.csv'
    
    context = ssl._create_unverified_context() # To aviod ssl error
    bd_srag_url = f'https://opendatasus.saude.gov.br/dataset/bd-srag-{year}'
    html_page = urlopen(bd_srag_url, context=context)
    soup = BeautifulSoup(html_page, features="lxml")
    for link in soup.findAll('a'):
        url = link.get('href')
        (filename, ext) = os.path.splitext(url)
        if ext.lower() == '.csv':
            srag_url = url
            print(f'\nCsv file found at <{bd_srag_url}>')
    
    return srag_url



def get_srag_data(years=[2021],update=True,save_local=True,treat=True,selected_columns='BASIC',aditional_columns=[]):
    
    sep = ';'
    quotechar = '"'
    frames = []
    for year in years:
        fname = f'data/opendatasus/INFLUD{year}.csv'
        if os.path.isfile(fname) and not update:
            print(f'\nReading OpenDataSus from local file <{fname}>. If you prefer to download last version, set "update=True".\n')
            df = pd.read_csv(fname,dtype=object)
        else:
            url = get_last_bd_srag_csv_url(year)
            print(f'\nDownloading from <{url}> ... ', end='')
            df = pd.read_csv(url,sep=sep,quotechar=quotechar,dtype=object)
            if save_local:
                df.to_csv(fname,index=False)
            print('complete!\n')
        frames.append(df)
    
    df = pd.concat(frames)
    if treat:
        df = treat_srag_data(df,selected_columns,aditional_columns)
    
    return df

def get_cities_states_dictionaries():
    ''' Returns 2 dictionaries: 
    1. cities_dict - city code (6 dig): city name
    2. states_dict - state code (2 dig): state name
    '''
    fname = 'data/IBGE/RELATORIO_DTB_BRASIL_MUNICIPIO.ods'
    df = pd.read_excel(fname,dtype=object)
    df['cod_municipio'] = df['Código Município Completo'].str[:6]
    cities_dict = df.set_index('cod_municipio')['Nome_Município'].to_dict()
    states_dict = df[['UF','Nome_UF']].groupby('UF').first()['Nome_UF'].to_dict()
    return cities_dict, states_dict

def treat_srag_data(df,selected_columns='BASIC',aditional_columns=[]):
    "Select columns, set types and replace values."
    df = df.copy()
    date_cols = ['DT_SIN_PRI','DT_EVOLUCA','DT_NASC','DT_ENTUTI']
    cities_cols = ['CO_MUN_RES','CO_MU_INTE','CO_MUN_NOT']
#     str_cols = cities_cols
    
    if selected_columns != 'ALL':
        basic_cols = date_cols + cities_cols
        basic_cols += ['SEM_PRI', 'EVOLUCAO', 'CLASSI_FIN','CLASSI_OUT',                
                      'NU_IDADE_N','CS_RACA', 'CS_ESCOL_N', 'CS_SEXO',
#                       'ID_MN_RESI','ID_MN_ITE','ID_MUNICIP',
                      'UTI', 'SUPORT_VEN']
    
        if selected_columns == 'BASIC':
            cols = basic_cols
        else:
            cols = basic_cols + list(aditional_columns)
        df = df[cols]
        
    df_cols = df.columns
    numeric_cols = ['SEM_PRI','NU_IDADE_N']
#     numeric_cols = list(set(df_cols) - set(date_cols) - set(str_cols))
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce',dayfirst=True)

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    first_date_2021 = pd.to_datetime('2021-01-03')
    mask = (df.DT_SIN_PRI >= first_date_2021)
#     mask = df.DT_SIN_PRI.dt.year == 2021
    df['SEM_PRI_ABS'] = df['SEM_PRI']
    df.loc[mask,'SEM_PRI_ABS'] = df.loc[mask,'SEM_PRI'] + 53
#     + (df.DT_SIN_PRI.dt.year - 2020) * 53
    
    cities_dict,states_dict = get_cities_states_dictionaries()
    regions_dict = {'1':'Norte',
                    '2':'Nordeste',
                    '3':'Sudeste',
                    '4':'Sul',
                    '5':'Centro-Oeste' }
    
    for col in cities_cols:
#         city_name_col = col[3:]
        sufix_index = col.index('_',3)
        city_name_col = 'MUN' + col[sufix_index:]
        state_name_col = 'UF' + col[sufix_index:]
        region_name_col = 'REGIAO' + col[sufix_index:]
        df[city_name_col] = df[col].apply(lambda x: cities_dict.get(x,'n.d.'))
        df[state_name_col] = df[col].str[:2].apply(lambda x: states_dict.get(x,'n.d.'))
        df[region_name_col] = df[col].str[0].apply(lambda x: regions_dict.get(x,'n.d.'))
        
    evolucao_dict = {'1':'cura',
                     '2':'obito',
                     '3':'obito_outras_causas',
                     '9':'ignorado' }
    classi_fin_dict = {'1':'cura',
                     '2':'obito',
                     '3':'obito_outras_causas',
                     '9':'ignorado' }

    df['EVOLUCAO'] = df['EVOLUCAO'].apply(lambda x: evolucao_dict.get(x,'n.d.'))
    df['CLASSI_FIN'] = df['CLASSI_FIN'].apply(lambda x: classi_fin_dict.get(x,'n.d.'))
    dict_cols = ['EVOLUCAO','CLASSI_FIN']
    
    other_cols = list(set(df_cols) - set(date_cols) - set(numeric_cols) - set(cities_cols) - set(dict_cols))
    df[other_cols] = df[other_cols].fillna('n.d.')
#     for col in str_cols:
#         df[col] = df[col].fillna('n.d.')
    

        
    return df

def get_pivot_data(df_srag,index_cols=[],columns_cols=[],values_cols='',last_week=999,selection_dict={}):
    
    df = df_srag.copy()

    for column,value in selection_dict.items():
        df = df.query(f'{column} == "{value}"')
    
    
    df = df.groupby(by=index_cols + columns_cols)[values_cols].count().reset_index()
    df = df.pivot(index=index_cols,columns=columns_cols,values=values_cols).fillna(0)
    if len(index_cols) == 2:
        i = 0    
        index_col_1 = index_cols[i]
        index_col_2 = index_cols[i+1]
        df1 = df.reset_index().set_index(index_col_1)
        df2 = df1.groupby(by=index_col_1).sum()
        df2[index_col_2] = 'TODOS'
        df = pd.concat([df1,df2])
        
    df['total'] = df.sum(axis=1)
    df = df.query('SEM_PRI_ABS <= @last_week').reset_index()
    return df


In [3]:
#hide

df_original = get_srag_data(years=[2020,2021],update=False,treat=False,save_local=True)


Reading OpenDataSus from local file <data/opendatasus/INFLUD2020.csv>. If you prefer to download last version, set "update=True".


Reading OpenDataSus from local file <data/opendatasus/INFLUD2021.csv>. If you prefer to download last version, set "update=True".



In [4]:
#hide

print(df_original.shape)
assert df_original.shape[0]>=1608416
assert df_original.shape[1]==154

(1608416, 154)


In [5]:
#hide

df_srag = treat_srag_data(df_original)

In [6]:
#hide

print(df_srag.shape)
assert df_srag.shape[0]==df_original.shape[0]
assert df_srag.shape[1]==27

(1608416, 27)


In [7]:
#collapse_output

df_srag.dtypes

DT_SIN_PRI     datetime64[ns]
DT_EVOLUCA     datetime64[ns]
DT_NASC        datetime64[ns]
DT_ENTUTI      datetime64[ns]
CO_MUN_RES             object
CO_MU_INTE             object
CO_MUN_NOT             object
SEM_PRI                 int64
EVOLUCAO               object
CLASSI_FIN             object
CLASSI_OUT             object
NU_IDADE_N              int64
CS_RACA                object
CS_ESCOL_N             object
CS_SEXO                object
UTI                    object
SUPORT_VEN             object
SEM_PRI_ABS             int64
MUN_RES                object
UF_RES                 object
REGIAO_RES             object
MUN_INTE               object
UF_INTE                object
REGIAO_INTE            object
MUN_NOT                object
UF_NOT                 object
REGIAO_NOT             object
dtype: object

In [8]:
#collapse

index_cols = ['SEM_PRI_ABS','UF_RES']
# index_cols = ['SEM_PRI_ABS']
columns_cols = ['EVOLUCAO']
values_cols = 'DT_SIN_PRI'
last_week = 62

df = get_pivot_data(df_srag,index_cols,columns_cols,values_cols,last_week)
df['tx_obito_final'] = df['obito'] / (df['obito']  + df['cura'] )
df

EVOLUCAO,SEM_PRI_ABS,UF_RES,cura,ignorado,n.d.,obito,obito_outras_causas,total,tx_obito_final
0,1,Acre,1.0,0.0,2.0,0.0,0.0,3.0,0.000000
1,1,Alagoas,0.0,1.0,0.0,0.0,0.0,1.0,NaN
2,1,Amazonas,8.0,0.0,1.0,3.0,0.0,12.0,0.272727
3,1,Bahia,11.0,0.0,3.0,1.0,0.0,15.0,0.083333
4,1,Ceará,11.0,0.0,3.0,1.0,0.0,15.0,0.083333
...,...,...,...,...,...,...,...,...,...
1767,58,TODOS,14912.0,980.0,9198.0,7696.0,365.0,33151.0,0.340410
1768,59,TODOS,15276.0,1067.0,10885.0,8326.0,321.0,35875.0,0.352767
1769,60,TODOS,17039.0,1111.0,15156.0,10607.0,332.0,44245.0,0.383672
1770,61,TODOS,15484.0,1081.0,17755.0,10726.0,287.0,45333.0,0.409233


In [9]:
#hide
# fig, ax1 = plt.subplots()
# color = 'tab:blue'
# ax1.set_ylabel('Total de casos', color=color)
# df_cases.plot( y = 'total', color=color, ax=ax1)
# ax1.set_xlabel('Semana Epidemiológia')
# ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# color = 'tab:red'
# ax2.set_ylabel('Taxa de óbitos', color=color)
# y_col = 'tx_obito_final'
# df_cases.plot( y=y_col, color=color, ax=ax2)
# ax2.tick_params(axis='y', labelcolor=color)

# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()

In [17]:
#colapse

# single-value selection over [Major_Genre, MPAA_Rating] pairs
# use specific hard-wired values as the initial selected values

UF_RES_list = df.UF_RES.unique().tolist()
UF_RES_list.sort()

selection = alt.selection_single(
    name='Select',
    fields=['UF_RES'],
    init={'UF_RES': 'Bahia'},
    bind={'UF_RES': alt.binding_select(options=UF_RES_list)}
#     fields=['Estado', 'Cidade'],
#     init={'Estado': 'Bahia', 'Cidade': 'Salvador'},
#     bind={'Estado': alt.binding_select(options=genres), 'Cidade': alt.binding_radio(options=mpaa)}
)
  
# scatter plot, modify opacity based on selection
alt.Chart(df).mark_line().add_selection(
    selection
).encode(
    x='SEM_PRI_ABS',
    y='tx_obito_final',
    color='UF_RES:N',
    tooltip='UF_RES',
    opacity=alt.condition(selection, alt.value(1.0), alt.value(0.10))
).properties(
    width=500,
    height=400
)

alt.Chart(...)